In [23]:
#mounting drive 
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [24]:
#train data reading
import pandas as pd
X_train=pd.read_csv('/drive/My Drive/project/dataset/new/train_X.csv',index_col=0)
y_train=pd.read_csv('/drive/My Drive/project/dataset/new/train_y.csv',index_col=0)

In [25]:
#importing necessary files 
from datetime import datetime
from operator import attrgetter
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from scipy.spatial import distance
import random
import pandas as pd
import math

In [26]:
#initializing required variables 
iters=10
pop=10
Field_S=[]
#parameters with PF algorithm
if pop/2 < 5:
    q_max=5
q_max=math.ceil(pop/2)
fit_t=0
Field=[] # Field is the entire population of PF algo
#Spider Monkey Algos params
LLL=5       #local leader limit
GLL=5       #global leader limit
group=4
cr=0.1
part=1
max_part=10
GlobalLimitCount=0
GlobalMax=0
GlobalLeaderPosition=bin(0)
GpMonkey=[]
gpoint =[[0 for i in range(2)] for j in range(group)]
LocalMax=[0 for i in range(5)]
LocalLimitCount=[0 for i in range(5)]
LocalLeaderPosition=[bin(0) for i in range(5)]
initial=[]
MaxM=[]
MaxP=[]
feat=6 #no of features

In [27]:
#PFA initialization
class Paddy():
    """docstring for Paddy"""
    def __init__(self, p, f):
        self.position = p
        self.fitness= f
        self.seed=0
        self.neighbour=1

def initializePaddy(pop, Field):
    for i in range(pop):
        m=initial[i]
        fit=calcFitness(m)
        F=Paddy(m, fit)
        Field.append(F)

def setPaddy(m):
    f=[]
    for i in range(len(m)):
        f.append(Paddy(m[i].position,m[i].fitness))
    return f

def addPaddy(m,M):
    fit=calcFitness(m)
    F=Paddy(m,fit)
    M.append(F)
    return M

In [28]:
#Phases of PFA
def selection(Field):
    global fit_t
    Sort_Field=sorted(Field, key=attrgetter('fitness'))
    print(len(Sort_Field))
    n0=int(len(Sort_Field)/3)
    if n0==0:
        n0=1
    n=len(Sort_Field)-n0
    fit_t=Sort_Field[n].fitness
    new_field=[]
    for i in range(len(Field)):
        if(Field[i].fitness <= fit_t):
            print(Field[i].fitness)
            print('>', fit_t)
            new_field.append(Field[i])
    print(len(new_field))
    return new_field
    #print(Sort_Field[-1].fitness)

def seeding(M):
    global fit_t
    y_max=getBestPaddy(M).fitness
    for i in range(len(M)):
        s=int(q_max*((M[i].fitness - fit_t)/(y_max - fit_t +1)))
        M[i].seed=s
        #print("Q_max",q_max)
        #print(s)

def pollination(M):
    for i in range(len(M)):
        temp=M[i]
        s01=slice(2,len(M[i].position))
        for j in range(i+1,len(M)-1):
            s02=slice(2,len(M[j].position))
            t1=math.fabs(distance.euclidean(int(temp.position[s01],2),int(M[j].position[s02],2))-(2**32))
            if(t1>2**30):
                temp.neighbour+=1
                M[j].neighbour+=1
    max_n=getMaxNeighbour(M)
    for i in range(len(M)):
        temp=(M[i].neighbour / max_n) - 1
        u=math.exp(temp)
        M[i].seed=int(M[i].seed * u)
    for i in range(len(M)):
        for j in range(M[i].seed):
            a=list(M[i].position)
            for j in range(random.randint(1,feat)):
                temp=random.randint(2,len(a)-1)
                if(a[temp]=='0'):
                    a[temp]='1'
                else:
                    a[temp]='0'
            str=""
            str1=str.join(a)
            print(str1)
            M=addPaddy(str1,M)
    return M

def getBestPaddy(P):
    L = sorted(P, key=attrgetter('fitness'))
    return L[-1]

def getMaxNeighbour(P):
    L=sorted(P,key=attrgetter('neighbour'))
    return L[-1].neighbour


In [29]:
#initializing SMO
class Monkey():
    def __init__(self,pos,fit):
        self.position=pos
        self.fitness=fit
        self.prob=0

def setMonkey(m):
    global gpoint,GlobalMax,GlobalLeaderPosition,GlobalLimitCount,LocalMax,LocalLimitCount,LocalLeaderPosition,group,GpMonkey
    f=[]
    for i in range(len(m)):
        f.append(Monkey(m[i].position,m[i].fitness))
    L = getBestMonkey(f)
    GlobalMax=L.fitness
    GlobalLeaderPosition=L.position
    GlobalLimitCount=0
    for k in range(group):
        print("Init for group",k)
        LocalMax[k]=f[gpoint[k][0]].fitness
        LocalLimitCount[k]=0
        LocalLeaderPosition[k]=f[gpoint[k][0]].position

    return f

def initializeMon(pop, GpMonkey):
    global gpoint,GlobalMax,GlobalLeaderPosition,GlobalLimitCount,LocalMax,LocalLimitCount,LocalLeaderPosition,group
    for i in range(pop):
        m=initial[i]
        fit=calcFitness(m)
        Mon=Monkey(m, fit)
        GpMonkey.append(Mon)
        print("GPM",i,"fit",fit)
    L = getBestMonkey(GpMonkey)
    GlobalMax=L.fitness
    GlobalLeaderPosition=L.position
    GlobalLimitCount=0
    create_group()
    for k in range(group):
        print("Init for group",k)
        LocalMax[k]=GpMonkey[gpoint[k][0]].fitness
        LocalLimitCount[k]=0
        LocalLeaderPosition[k]=GpMonkey[gpoint[k][0]].position

def CalculateProbabilities():
    global GpMonkey,pop
    print("Calculating probabilities")
    maxfit=getBestMonkey(GpMonkey).fitness
    if(maxfit==0):
      maxfit=0.1
    for i in range(pop):
        GpMonkey[i].prob=(0.9 * (GpMonkey[i].fitness/maxfit))+0.1

def create_group():
    global pop,group,part
    print("Creating group")
    g=0
    lo=0
    while(lo < pop) and (g< group):
        hi= lo+int(pop/part)
        gpoint[g][0]=lo
        gpoint[g][1]=hi
        if((pop-hi)<(int(pop/part))):
            gpoint[g][1]=(pop-1)
        g=g+1
        lo=hi+1
    group = g

def LocalLearning():
    print("Local Learning")
    OldMax=np.zeros(group)
    for k in range(group):
        OldMax[k]=LocalMax[k]

    for k in range(group):
        i=gpoint[k][0]
        while(i<gpoint[k][1] and i< pop):
            print(i)
            print(LocalMax[k])
            if(GpMonkey[i].fitness > LocalMax[k]):
                LocalMax[k]=GpMonkey[i].fitness
                LocalLeaderPosition[k]=GpMonkey[i].position
            i=i+1
    
    for k in range(group):
        if(math.fabs(OldMax[k]-LocalMax[k])<1.0e-5):
            LocalLimitCount[k]=LocalLimitCount[k]+1
        else:
            LocalLimitCount[k]=0
        
def GlobalLearning():
    global GlobalMax,GlobalLeaderPosition,GlobalLimitCount
    print("Global Learning")
    G_trail=GlobalMax
    GM=getBestMonkey(GpMonkey)
    GlobalMax=GM.fitness
    GlobalLeaderPosition=GM.position
    if(math.fabs(G_trail-GlobalMax)<1.0e-5):
        GlobalLimitCount=GlobalLimitCount+1
    else:
        GlobalLimitCount=0
    print("GlobalLimitCount",GlobalLimitCount)

def LocalLeaderPhase(k):
    global cr
    print("Local Leader phase",k)
    lo=gpoint[k][0]
    hi=gpoint[k][1]
    i=lo
    print(i)
    while(i < hi):
        while True:
            print(hi)
            print(lo)
            PopRand=int((random.random()*(hi-lo)+lo))
            print("pop rand")
            print(PopRand)
            if(PopRand != i):
                break
            else:
                if(hi-lo==1):
                    break

        sobject=slice(2,len(GpMonkey[i].position))
        gpmp=int(GpMonkey[i].position[sobject],2)
        if(random.random() >= cr):
            so1=slice(2,len((LocalLeaderPosition[k])))
            sorr=slice(2,len(GpMonkey[PopRand].position))
            llpk=int(LocalLeaderPosition[k][so1],2)
            gppr=int(GpMonkey[PopRand].position[sorr],2)
            new_position=int(math.fabs(gpmp+(llpk-gpmp)
            *(random.random())+(gppr-gpmp)*(random.random()-0.5)*2))
        else:
            new_position=gpmp
        
        new_fitness=calcFitness(bin(new_position))
        if(new_fitness < GpMonkey[i].fitness):
            GpMonkey[i].position=bin(new_position)
            GpMonkey[i].fitness=new_fitness
        i=i+1

def GlobalLeaderPhase(k):
    print("Global Leader Phase",k)
    lo=int(gpoint[k][0])
    hi=int(gpoint[k][1])
    i=lo
    l=lo
    while(l<hi):
        if( random.random() < GpMonkey[i].prob):
            l=l+1
            while True:
                PopRand=int(random.random()*(hi-lo)+lo)
                if (PopRand != i):
                    break
                else:
                    if(hi-lo==1):
                        break
            sobject=slice(2,len(GpMonkey[i].position))
            gpmp=int(GpMonkey[i].position[sobject],2)
            so1=slice(2,len(GlobalLeaderPosition))
            sorr=slice(2,len(GpMonkey[PopRand].position))
            glpk=int(GlobalLeaderPosition[so1],2)
            gppr=int(GpMonkey[PopRand].position[sorr],2)
            new_position=int(math.fabs(gpmp+(glpk-gpmp)
            *(random.random())+(gppr-gpmp)*(random.random()-0.5)*2))
            new_fitness=calcFitness(bin(new_position))
            if(new_fitness < GpMonkey[i].fitness):
                GpMonkey[i].position=bin(new_position)
                GpMonkey[i].fitness=new_fitness
        i=i+1
        if(i==hi):
            i=lo

def GlobalLeaderDecision():
    global GlobalLimitCount,GLL,part,max_part
    print("GlobalLeaderDecision")
    print("Global Limit Count",GlobalLimitCount)
    print("GLL",GLL)
    if(GlobalLimitCount > GLL):
        GlobalLimitCount=0
        if(part<max_part):
            part=part+1
            create_group()
            LocalLearning()
        else:
            part=1
            create_group()
            LocalLearning()

def LocalLeaderDecision():
    global LLL
    print("LocalLeaderDecision")
    for k in range(group):
        if(LocalLimitCount[k]>LLL):
            i=gpoint[k][0]
            while(i<=gpoint[k][1]):
                sobject=slice(2,len(GpMonkey[i].position))
                gpmp=int(GpMonkey[i].position[sobject],2)
                so1=slice(2,len(GlobalLeaderPosition))
                glpk=int(GlobalLeaderPosition[so1],2)
                so1=slice(2,len(LocalLeaderPosition[k]))
                llpk=int(LocalLeaderPosition[k][so1],2)
                if(random.random()>=cr):
                    GpMonkey[i].position=bin(random.randrange(1,(2**(feat+1))-1))
                else:
                    GpMonkey[i].position=bin(int(math.fabs(int(gpmp+(glpk-gpmp)
                    *random.random()+(glpk-llpk)*random.random()))))
                
                GpMonkey[i].fitness=calcFitness(GpMonkey[i].position)
                i+=1
            LocalLimitCount[k]=0

def getBestMonkey(GpMonkey):
    print("GPM fitness",GpMonkey[0].fitness)
    L = sorted(GpMonkey, key=attrgetter('fitness'))
    return L[0]

In [30]:
def Monkeyexe():
    global cr,GpMonkey
    GlobalLearning()
    LocalLearning()
    create_group()
    for i in range(2):
        for k in range(group):
            LocalLeaderPhase(k)
        CalculateProbabilities()
        for k in range(group):
            print("group",k)
            GlobalLeaderPhase(k)
        GlobalLearning()
        LocalLearning()
        LocalLeaderDecision()
        GlobalLeaderDecision()
        cr = cr + (0.4/iters)
        GpMonkey=sorted(GpMonkey, key=attrgetter('fitness'))
        MaxM.append(Monkey(GlobalLeaderPosition,GlobalMax))


In [31]:
def Paddyexe():
    global Field_S, Field
    for i in range(2):
        print(" calling selection")
        Field_S=selection(Field)
        print("calling seeding")
        seeding(Field_S)
        print(" calling pollination")
        Field_S=pollination(Field_S)
        Field_S=sorted(Field_S, key=attrgetter('fitness'))
        max_p=getBestPaddy(Field_S)
        MaxP.append(max_p)

In [32]:
import tensorflow
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError
import statistics

def calcFitness(s):
    global train_set,test_set,traint,testt
    print(s)
    p=pd.DataFrame()
    q=pd.DataFrame()
    count=0
    x=(feat+2)-len(s)
    if(x<0):
        return 0
    while(x>0):
        count=count+1
        x=x-1
    for i , digit in enumerate(s): #selecting columns with value 1
       if(i>1):
           #print("digit",digit)
           if(digit=='1'):
                count+=1
                #m=pd.DataFrame(train_set.iloc[:,count-1])
                #o=pd.DataFrame(test_set.iloc[:,count-1])
                m=pd.DataFrame(X_train.iloc[:,count-1])
                #o=pd.DataFrame(test_set.iloc[:,count-1])
                p=pd.concat([p,m],axis=1)
                #q=pd.concat([q,o],axis=1)
           else:
                count+=1
    print(p.shape) 
    p1=p.to_numpy()
    ptest=q.to_numpy()
    #trat=traint.to_numpy(dtype=int)
    trat=y_train.to_numpy(dtype=int)
    #tet=testt.to_numpy(dtype=int)
    #print("Train shape",p1.shape)
    #print("Test shape",ptest.shape)
    model = build_model_using_sequential()
    msle = MeanSquaredLogarithmicError()
    model.compile(
    loss=msle, 
    optimizer=Adam(learning_rate=learning_rate), 
    metrics=[msle]
    )
    # train the model
    history = model.fit(
    p.values, 
    y_train.values, 
    epochs=10, 
    batch_size=64,
    validation_split=0.2)
    #from sklearn.ensemble import RandomForestClassifier
    #clf = RandomForestClassifier()
    #clf.fit(p1,trat)
    #y_pred=clf.predict(ptest)
    #mae = np.abs(tet-y_pred)
    #score=mae.mean()
    #score=accuracy_score(tet,y_pred, normalize=True)
    score=statistics.mean(history.history['val_mean_squared_logarithmic_error'])
    print(score)
    return score

  #input layer , hidden layer 1 , 2,3,output layer 

hidden_units1 = 20
hidden_units2 = 40
hidden_units3 = 30
learning_rate = 0.01
# Creating model using the Sequential in tensorflow
def build_model_using_sequential():
      model = Sequential([
        Dense(hidden_units1, kernel_initializer='normal', activation='relu'),
        Dropout(0.2),
        Dense(hidden_units2, kernel_initializer='normal', activation='relu'),
        Dropout(0.2),
        Dense(hidden_units3, kernel_initializer='normal', activation='relu'),
        Dense(1, kernel_initializer='normal', activation='linear')
      ])
      return model

In [33]:
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from operator import attrgetter
from scipy.spatial import distance

#initializing population
def GenInitPop():
    print("GetInit pop function ")
    a=[0 for i in range(feat)]
    for h in range(pop):
        for i in range(0,feat):
            a[i]=random.randint(0,1)
        str1="0b"
        for x in a:
            str1+=str(x)
        initial.append(str1)   #initial is a list with "pop,no of feature string as same as pop"
    print(initial)

def Fight(sm,pf):
    print("I am from fight")
    newpop=[]
#    limit=0
#    if(len(pf)<len(sm)):
#        limit=len(pf)
#    else:
#        limit=len(sm)
    for i in range(pop):
        if i< (len(pf)):
            print(i)
            if(sm[i].fitness < pf[i].fitness):
                newpop.append(sm[i])
            else:
                newpop.append(pf[i])
            print(newpop[i])
        else:
            newpop.append(sm[i])
    return newpop
    
def main():
    global Field, GpMonkey,pop
    GenInitPop()
    print("Initializing monkey")
    initializeMon(pop,GpMonkey)
    print("paddy initializing")
    initializePaddy(pop,Field)
    for i in range(iters):
        print("monkey exec ")
        Monkeyexe()
        print("paddy exec ")
        Paddyexe()
        print("fight exec ")
        New_pop=Fight(GpMonkey,Field_S)
        pop=len(New_pop)
        GpMonkey=[]
        Field=[]
        GpMonkey=setMonkey(New_pop)
        Field=setPaddy(New_pop)

    for i in range(len(MaxM)):
        print("iter",i+1)
        print("Best Position",MaxM[i].position)
        print("Best Fitness",MaxM[i].fitness)
    for i in range(len(MaxP)):
        print("iter",i+1)
        print("Best Position",MaxP[i].position)
        print("Best Fitness",MaxP[i].fitness)
    asdd=getBestMonkey(MaxM)
    print("Overall Best position",asdd.position)
    print("Overall Best fitness",asdd.fitness)
if __name__=="__main__":
    main()

GetInit pop function 
['0b100000', '0b110100', '0b111111', '0b010000', '0b101111', '0b010000', '0b110110', '0b110011', '0b011001', '0b011001']
Initializing monkey
0b100000
(7975, 1)
Epoch 1/10
100/100 [==============================] - 1s 4ms/step - loss: 41.1838 - mean_squared_logarithmic_error: 41.1075 - val_loss: 12.8669 - val_mean_squared_logarithmic_error: 12.8670
Epoch 2/10
100/100 [==============================] - 0s 3ms/step - loss: 8.8303 - mean_squared_logarithmic_error: 8.8197 - val_loss: 5.1275 - val_mean_squared_logarithmic_error: 5.1277
Epoch 3/10
100/100 [==============================] - 0s 3ms/step - loss: 3.8644 - mean_squared_logarithmic_error: 3.8619 - val_loss: 2.3779 - val_mean_squared_logarithmic_error: 2.3783
Epoch 4/10
100/100 [==============================] - 0s 3ms/step - loss: 2.2536 - mean_squared_logarithmic_error: 2.2536 - val_loss: 1.5527 - val_mean_squared_logarithmic_error: 1.5531
Epoch 5/10
100/100 [==============================] - 0s 3ms/step - lo